In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer

ImportError: cannot import name 'IterativeImputer' from 'sklearn.impute' (/home/lcp87/anaconda3/lib/python3.9/site-packages/sklearn/impute/__init__.py)

In [8]:
df = pd.read_csv("pairs_limpieza.csv", index_col = 0)
df.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,species_,injury,fecha_limpia,fatal_clean,sexo
0,1800.00.00,1800,Unprovoked,seychelles,St. Anne,NaN,a corsair's boat was overturned,NaN,NaN,NaN,NaN,"FATAL, all onboard were killed by sharks",NaN,Y,F
1,1797.05.28.R,1797,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,FATAL,May,Y,NaN


Vamos a ver cuántos nulos tenemos en nuestro df

In [9]:
df.isnull().sum()

case_number        1
year               0
type               4
country           50
area             454
location         539
activity         542
name             209
age             2820
time            3340
species_        2831
injury            28
fecha_limpia     526
fatal_clean      607
sexo             564
dtype: int64

# Ejercicio 1. Eliminar nulos

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

Sabiendo el número de nulos que tiene la columna age, vamos a prodoceder a reemplazarlos.

In [10]:
df["age"].unique()

array([nan, 'young', '14', '19', '20', '15', '21', '27', '36', '25', '6',
       '16', '50', '13 or 14', '57', '11', '48', '18', '52', '12', '32',
       '10', '34', '30', '60', '33', '29', '54', '41', '37', '56', '69',
       '38', '55', '35', '46', '45', '40s', '28', '24', '26', '49', '22',
       '7', '31', '17', '40', '13', '42', '3', '8', '82', '73', '20s',
       '68', '51', '39', '58', 'Teen', '47', '61', '65', '66', '43',
       '60s', '9', '72', '59', '64', '23', '71', '44', '62', '63', '70',
       '18 months', '53', '30s', '50s', 'teen', '77', '74', '28 & 26',
       '5', '86', '18 or 20', '12 or 13', '46 & 34', '28, 23 & 30',
       'Teens', '36 & 26', '8 or 10', '84', '\xa0 ', ' ', '30 or 36',
       '6½', '21 & ?', '75', '33 or 37', 'mid-30s', '23 & 20', ' 30',
       '7      &    31', ' 28', '20?', "60's", '32 & 30', '16 to 18',
       '87', '67', 'Elderly', 'mid-20s', 'Ca. 33', '74 ', '45 ',
       '21 or 26', '20 ', '>50', '18 to 22', 'adult', '9 & 12', '? & 19',
     

Antes de proceder a elegir el argumento strategy de nnuestro SimpleImputer, vamos a hacer un pequeño análisis de cuál estadístico sería más representativo. 

Hacemmos gráfica (vemos si está sesgada a la derecha, si no, o si es uan distribución normal... ahí justificamos si utilizamos comos strategy media o mdiana u otro.) Se supone que hemos elegido Mediana

In [5]:
imputer = SimpleImputer(df["age"], missing_values= np.nan, strategy = "median" )                # asegurarnos de que nuestros NANs son np.nan y si no poner otro miising_values.

/home/lcp87/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass missing_values=0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
6279    NaN
6280    NaN
6281    NaN
6282    NaN
6283    NaN
Name: age, Length: 6284, dtype: object as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [6]:
imputer = imputer.fit(df[["age"]])

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'young'

In [ ]:
df["age_sin_nulos"] = imputer.transform(df[["age"]])

In [ ]:
df["age_sin_nulos"]

In [ ]:
df["age_sin_nulos"].isnull().sum()

Observamos que la columna age ha cambiado los nulos por el valor de la mediana de esa misma columna, por lo tanto ahora la columna age no tiene valores nulos.

- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 ** Pista** 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [19]:
imputer = SimpleImputer(missing_values= np.nan, strategy = "most_frequent" )     

In [20]:
imputer = imputer.fit(df[["sexo"]])

In [21]:
df["sexo_sin_nulos"] = imputer.transform(df[["sexo"]])

In [22]:
df["sexo_sin_nulos"]

0       F
1       M
2       M
3       F
4       M
       ..
6279    M
6280    M
6281    M
6282    M
6283    M
Name: sexo_sin_nulos, Length: 6284, dtype: object

In [23]:
df["sexo_sin_nulos"].isnull().sum()

0

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [24]:
imputer = SimpleImputer(missing_values= np.nan, strategy = "most_frequent")

In [25]:
imputer = imputer.fit(df[["type"]])

In [26]:
df["type_sin_nulos"] = imputer.transform(df[["type"]])

In [27]:
df["type_sin_nulos"].isnull().sum()

0

Comprobamos que ya no existen valores nulos en la columna type. Todos los nulos se han reemplazado por el valor de la moda.

- Utilizad el método **KNN** **Imputer** para reemplazar todos los valores nulos de las columnas numéricas.

In [43]:
df_numericas = df.select_dtypes(include = np.number)                        #En este DF no aparecen todas las columnas numéricas, aparecerán cuando los archivos estén limpios.(asegurate que aparezca AGE)
                                                                            #cambiar el tipo de col time a (datetime64)
df_numericas.head(2)

,year
0,1800
1,1797


In [47]:
imputerKNN = KNNImputer(n_neightbors = 5)
imputerKNN.fit(df_numericas)

TypeError: __init__() got an unexpected keyword argument 'n_neightbors'

In [45]:
imputerKNN.fit(df_numericas)

NameError: name 'imputerKNN' is not defined

In [46]:
numericas_KNN = imputerKNN.transform(df_numericas)


NameError: name 'imputerKNN' is not defined

asdasd

 - Utilizad el método **Iterative Imputer** para reemplazar todos los valores nulos de las columnas numéricas.

In [12]:
imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')


NameError: name 'IterativeImputer' is not defined

In [13]:
imputer.fit(df_numericas)

NameError: name 'df_numericas' is not defined

In [14]:
imputer.transform(df_numericas)

NameError: name 'df_numericas' is not defined

- ¿Podríais explicar que diferencia hay entre estos dos últimos métodos?

Ambos son métodos para reemplazar nulos. Las diferencias entre IterativeImputer y KNNImputer es que InterativeImputer los reemplaza en base a las columnas del df y el KNNInputer en base a los vecinos más próximos. 


IterativeImputer elige un valor para los nulos en relación al df. Elegimos un estadístico (media, mediana...) y estima el valor a reemplazar en base a ese estadístico.

KNNImputer clasifica cada uno de los datos comparando ese dato y su similitud entre sus vecinos más próximos.

# Ejercicio 2. Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [ ]:
df.to_csv("pairs_limpieza.csv")